<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [123]:
!unzip '/content/drive/My Drive/jigsaw-toxic-comment-train.csv.zip'

Archive:  /content/drive/My Drive/jigsaw-toxic-comment-train.csv.zip
replace jigsaw-toxic-comment-train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from statsmodels.graphics.mosaicplot import mosaic
import statistics 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
import seaborn as sns

In [125]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#print(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [126]:
my_train = pd.read_csv('jigsaw-toxic-comment-train.csv')

In [166]:
my_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             223549 non-null  object
 1   comment_text   223549 non-null  object
 2   toxic          223549 non-null  int64 
 3   severe_toxic   223549 non-null  int64 
 4   obscene        223549 non-null  int64 
 5   threat         223549 non-null  int64 
 6   insult         223549 non-null  int64 
 7   identity_hate  223549 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 13.6+ MB


In [127]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
my_train.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the tools well. · talk """,0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0,0,0,0,0,0
8,00037261f536c51d,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contrary to those of DuLithgow,0,0,0,0,0,0


### **Анализ всех дополнительных колонок тональности (как проявляется тот или иной тип токсичности, как в данных это представлено, какие есть пограничные случаи)**

In [128]:
len(my_train.loc[my_train['toxic']==1])

21384

In [129]:
def max_tox_len(toxic_type):
  toxic_type_df = my_train.loc[my_train[toxic_type] == 1]
  len_list = [len(sent) for sent in toxic_type_df['comment_text']]
  return max(len_list)


print('toxic', max_tox_len('toxic'))
print('severe_toxic', max_tox_len('severe_toxic'))
print('obscene', max_tox_len('obscene'))
print('threat', max_tox_len('threat'))
print('insult', max_tox_len('insult'))
print('identity_hate', max_tox_len('identity_hate'))


toxic 5000
severe_toxic 5000
obscene 5000
threat 5000
insult 5000
identity_hate 5000


In [130]:
def max_none_tox_len(none_toxic_type):
  toxic_type_df = my_train.loc[my_train[none_toxic_type] == 0]
  len_list = [len(sent) for sent in toxic_type_df['comment_text']]
  return max(len_list)


print('toxic', max_none_tox_len('toxic'))
print('severe_toxic', max_none_tox_len('severe_toxic'))
print('obscene', max_none_tox_len('obscene'))
print('threat', max_none_tox_len('threat'))
print('insult', max_none_tox_len('insult'))
print('identity_hate', max_none_tox_len('identity_hate'))

toxic 5000
severe_toxic 5000
obscene 5000
threat 5000
insult 5000
identity_hate 5000


In [131]:
#Получается совсем не информативно, попробую измерить среднее

In [132]:
def mean_tox_len(toxic_type):
  toxic_type_df = my_train.loc[my_train[toxic_type] == 1]
  len_list = [len(sent) for sent in toxic_type_df['comment_text']]
  return statistics.mean(len_list)


print('toxic', mean_tox_len('toxic'))
print('severe_toxic', mean_tox_len('severe_toxic'))
print('obscene', mean_tox_len('obscene'))
print('threat', mean_tox_len('threat'))
print('insult', mean_tox_len('insult'))
print('identity_hate', mean_tox_len('identity_hate'))

toxic 280.60409652076316
severe_toxic 505.4566768603466
obscene 280.9517298187809
threat 316.9100145137881
insult 272.2064755838641
identity_hate 314.43882853094


In [133]:
#тут немного более информативно, severe_toxic явно выделяется, например. Попробую взять не-токсичные:

In [134]:
def mean_none_tox_len(none_toxic_type):
  toxic_type_df = my_train.loc[my_train[none_toxic_type] == 0]
  len_list = [len(sent) for sent in toxic_type_df['comment_text']]
  return statistics.mean(len_list)


print('toxic', mean_none_tox_len('toxic'))
print('severe_toxic', mean_none_tox_len('severe_toxic'))
print('obscene', mean_none_tox_len('obscene'))
print('threat', mean_none_tox_len('threat'))
print('insult', mean_none_tox_len('insult'))
print('identity_hate', mean_none_tox_len('identity_hate'))

toxic 402.6911779981698
severe_toxic 389.999386245583
obscene 397.33287608379965
threat 391.2418065153011
insult 397.34023416334895
identity_hate 391.7447929838506


In [135]:
#Попробую выделить списки слов, характерных для конкретных типов.

In [136]:
def tokenize(sents):
    words = []
    for sent in sents:
      for word in sent.split():
        words.append(word.strip(string.punctuation))
    words = [word for word in words if word]
    
    return words

In [137]:
#  это просто на будущее более удобное разделение данных по типам
toxic_df = my_train.loc[my_train['toxic'] == 1]
severe_toxic_df = my_train.loc[my_train['severe_toxic'] == 1]
obscene_df =  my_train.loc[my_train['obscene'] == 1]
threat_df = my_train.loc[my_train['threat'] == 1]
insult_df = my_train.loc[my_train['insult'] == 1]
identity_hate_df = my_train.loc[my_train['identity_hate'] == 1]

In [138]:
stoplist = stopwords.words('english')
for w in ['would', 'page', 'wikipedia', 'want', 'think', 'time']:
  stoplist.append(w)
'would' in stoplist

True

In [139]:
def freq_words(toxic_type, n):
  toxic_type_df = my_train.loc[my_train[toxic_type] == 1]
  length = len(toxic_type_df)
  counter = Counter(tokenize(toxic_type_df['comment_text']))
  frequent_dict = {key:val for key, val in counter.items() if val > length*n and \
                   key.lower() not in stoplist}

  return frequent_dict

In [140]:
freq_words('toxic', 0.1)


{'DIE': 2760,
 'FUCK': 5952,
 'NIGGER': 3397,
 'SUCK': 2675,
 'ass': 2240,
 'fuck': 6151,
 'fucking': 2766,
 'gay': 2192,
 'know': 2523,
 'like': 4149,
 'shit': 2971}

In [141]:
freq_words('severe_toxic', 0.1)

{'ADMINS': 200,
 'ANAL': 342,
 'ASS': 762,
 'Ass': 271,
 'BASTARD': 205,
 'BITCH': 644,
 'BITCH!!FUCK': 310,
 'BLOCK': 266,
 'BOT': 217,
 'Bitch': 293,
 'BunkSteve': 278,
 'COCK': 442,
 'COCKSUCKER': 494,
 'COCKSUCKING': 267,
 'CUNT': 819,
 'CUNTS': 241,
 'Chester': 261,
 'CriminalWar': 279,
 'DAMN': 377,
 'DICK': 331,
 'DIE': 1565,
 'DIE!FUK': 402,
 'DOG': 363,
 'EAT': 204,
 'FACK': 232,
 'FAGGOT': 605,
 'FAGS': 207,
 'FAT': 243,
 'FUCK': 3088,
 'FUCKER': 523,
 'FUCKING': 1084,
 'Fuck': 1051,
 'Fucking': 212,
 'GAY': 250,
 'GO': 223,
 'Go': 924,
 'HOMELAND': 229,
 'HUGE': 421,
 'KILL': 630,
 'MEXICANS': 358,
 'MOTHERFUCKER': 253,
 'MOTHERFUCKERDIE': 287,
 'MOTHJER': 489,
 'MUST': 562,
 'MarcolFuck': 260,
 'NIGGER': 808,
 'NIGGERJEW': 424,
 'NIGGERS': 393,
 'Nigger': 367,
 'Notrhbysouthbanof': 228,
 'OFFFUCK': 360,
 "Pro-Assad.Hanibal911You're": 345,
 'RAPE': 362,
 'SECURITYFUCK': 227,
 'SHIT': 1508,
 'SHUT': 364,
 'SUCK': 2253,
 'TRAITOR': 405,
 'U': 939,
 'USELESS': 224,
 'VANDAL': 4

In [142]:
freq_words('obscene', 0.1)

{'CUNT': 1440,
 'DICKS': 1223,
 'DIE': 1468,
 'FUCK': 5940,
 'FUCKING': 1592,
 'Fuck': 1961,
 'Go': 1240,
 'NIGGER': 2990,
 'SHIT': 1688,
 'SUCK': 2535,
 'U': 1319,
 'ass': 2121,
 'bitch': 1511,
 'fuck': 6073,
 'fucking': 2651,
 'get': 1248,
 'know': 1376,
 'like': 2061,
 'shit': 2623,
 'stupid': 1264,
 'u': 1599}

In [143]:
freq_words('threat', 0.1)

{'BAN': 131,
 'BLANK': 94,
 'BLOCK': 150,
 'DI': 90,
 'DIE': 1055,
 'DIE!FUK': 310,
 'Die': 89,
 'EDIE': 90,
 'FUCKIN': 82,
 'FUCKING': 74,
 'FoReVeR': 76,
 'FooL': 73,
 "I'm": 73,
 'JIM': 157,
 'KILL': 526,
 'LiVe': 76,
 'MUST': 478,
 'NIGGGERS': 123,
 'PaTHeTiC': 76,
 'RVV': 96,
 'ReSPeCT': 76,
 'SuPeRTR0LL': 149,
 'TALK': 101,
 'TRAITOR': 313,
 'VANDAL': 314,
 'WALES': 156,
 'ass': 753,
 'balls': 92,
 'bitch': 81,
 'decapitate': 84,
 'die': 120,
 'fuck': 102,
 'fucking': 176,
 'get': 84,
 'going': 383,
 'kill': 275,
 'like': 69,
 'murder': 109,
 'rip': 87,
 'shit': 96,
 'tiny': 83,
 'u': 70,
 'you.I': 81}

In [144]:
freq_words('insult', 0.1)

{'CUNT': 1134,
 'DIE': 1465,
 'FAT': 1565,
 'FUCK': 4238,
 'FUCKING': 1516,
 'Fuck': 1179,
 'Go': 1235,
 'JEW': 1246,
 'NIGGER': 3033,
 'SUCK': 2380,
 'U': 1279,
 'ass': 1882,
 'bitch': 1458,
 'faggot': 1718,
 'fuck': 4670,
 'fucking': 2252,
 'gay': 1154,
 'hi': 1345,
 'know': 1361,
 'like': 2052,
 'moron': 1474,
 'nigger': 1593,
 'shit': 2025,
 'stupid': 1447,
 'u': 1525}

In [145]:
freq_words('identity_hate', 0.1)

{'Ass': 271,
 'Bitch': 286,
 'BunkSteve': 278,
 'CUNT': 595,
 'DIE': 772,
 'FAGGOT': 555,
 'FAT': 1314,
 'FRANCE': 217,
 'FUCK': 696,
 'FUCKING': 449,
 'GAY': 681,
 'HATE': 228,
 'HUGE': 422,
 'JEW': 1245,
 'KILL': 313,
 'MEXICANS': 358,
 'NIGGER': 3098,
 'NIGGERJEW': 424,
 'NIGGERS': 1113,
 'Nigger': 507,
 'Please!Nigga': 383,
 'SUCK': 551,
 'TOMMY2010': 227,
 'ass': 327,
 'bitch': 324,
 'fag': 246,
 'faggot': 724,
 'fuck': 751,
 'fucking': 499,
 'gay': 1334,
 'like': 451,
 'nigga': 227,
 'niggas': 359,
 'nigger': 1594,
 'nigger!Dumbass': 215,
 'shit': 358,
 'stupid': 425,
 'sucks!!!!!!!!!!!U.S.A': 223,
 'u': 302}

In [146]:
#посмотрим, сколько частотных слов вообще в каком типе получилось

In [147]:
print('severe_toxic', len(freq_words('severe_toxic', 0.1)))
print('toxic', len(freq_words('toxic', 0.1)))
print('obscene', len(freq_words('obscene', 0.1)))
print('threat', len(freq_words('threat', 0.1)))
print('insult', len(freq_words('insult', 0.1)))
print('identity_hate', len(freq_words('identity_hate', 0.1)))

severe_toxic 101
toxic 11
obscene 21
threat 43
insult 25
identity_hate 39


In [148]:
#и как они пересекаются между собой. Я полагаю, что uppercase так же несет в себе информацию о тональности, 
# и полагаю upper-lowercase разными словами.

In [149]:
toxic_set = set(freq_words('toxic', 0.1).keys())
severe_toxic_set = set(freq_words('severe_toxic', 0.1).keys())
obscene_set = set(freq_words('obscene', 0.1).keys())
threat_set = set(freq_words('threat', 0.1).keys())
insult_set = set(freq_words('insult', 0.1).keys())
identity_hate_set = set(freq_words('identity_hate', 0.1).keys())                                                 
# как toxic пересекается с остальными

In [150]:
toxic_set.intersection(severe_toxic_set, obscene_set, threat_set, insult_set, identity_hate_set)

{'DIE', 'ass', 'fuck', 'fucking', 'like', 'shit'}

In [151]:
#И посмотрим, как они выделяются

In [152]:
print('severe_toxic_set', severe_toxic_set.difference(toxic_set, obscene_set, threat_set, insult_set, identity_hate_set))
print('toxic_set', toxic_set.difference(severe_toxic_set, obscene_set, threat_set, insult_set, identity_hate_set))
print('threat_set', threat_set.difference(toxic_set, severe_toxic_set, obscene_set, insult_set, identity_hate_set))
print('insult_set', insult_set.difference(toxic_set, obscene_set, threat_set, severe_toxic_set, identity_hate_set))
print('identity_hate_set', identity_hate_set.difference(toxic_set, obscene_set, threat_set, insult_set, identity_hate_set))
print('obscene_set', obscene_set.difference(toxic_set, identity_hate_set, threat_set, insult_set, identity_hate_set))


severe_toxic_set {'fat', 'fucked', 'COCKSUCKER', 'cunt', 'FACK', 'bitches.fuck', 'veggietales', 'dick', 'ancestryFuck-off-Jewish', 'BITCH', 'MOTHERFUCKER', 'MarcolFuck', 'Notrhbysouthbanof', 'BOT', 'asshole', 'BASTARD', 'bastard', 'sucks', 'GO', 'ASS', 'cock', 'ass!Fuck', 'COCKSUCKING', 'FAGS', 'EAT', 'COCK', 'suck', 'piece', "Pro-Assad.Hanibal911You're", 'MOTHJER', 'cocks', 'RAPE', 'CUNTS', "YOU!!'FUCK", 'dickhead', 'HOMELAND', 'little', 'DOG', 'BITCH!!FUCK', 'CriminalWar', 'DICK', 'yourselfgo', 'ADMINS', 'DAMN', 'OFFFUCK', 'Chester', 'pussy', 'USELESS', 'SECURITYFUCK', 'ANAL', 'FUCKER', 'fucksex', 'SHUT', 'Fucking', 'penis/////Small', 'MOTHERFUCKERDIE', 'mother'}
toxic_set set()
threat_set {'SuPeRTR0LL', 'decapitate', 'FooL', 'PaTHeTiC', 'RVV', 'die', 'you.I', 'EDIE', 'FoReVeR', 'LiVe', 'WALES', 'BLANK', 'NIGGGERS', 'TALK', 'DI', 'BAN', 'kill', 'FUCKIN', "I'm", 'ReSPeCT', 'murder', 'rip', 'JIM', 'Die', 'balls', 'tiny'}
insult_set {'moron'}
identity_hate_set set()
obscene_set {'DICKS'

In [153]:
#в severe_toxic просто больше слов, а вот threat можно заметить слова с верхним и нижним регистром в одном.
# Попробую их тоже рассматривать как признак  и посчитать.

In [154]:
re.findall(r'([A-Z]+[a-z]+[A-Z]+)|([a-z]+[A-Z]+[a-z]+)', 'HellO')

[('HellO', '')]

In [155]:
def upper_lower_words(toxic_type):
  len_list = []
  token_list = []
  toxic_type_df = my_train.loc[my_train[toxic_type] == 1]
  length = len(toxic_type_df)
  df_tokens = tokenize(toxic_type_df['comment_text'])
  for token in df_tokens:
    if len(re.findall(r'([A-Z]+[a-z]+[A-Z]+)|([a-z]+[A-Z]+[a-z]+)', token)) > 0:
      len_list.append(len(token))
      token_list.append(token)
  return len_list

In [156]:
#еще потом придумаю, но минималка какая-то вроде есть.
# len_list = [len(sents) for sents in my_train['comment_text']]
# punkt_len_list = [len(re.findall(r'[\.,?!)\";}\]\*:@\'\({\[]+', sents)) for sents in my_train['comment_text']]
# upper_len_list = [len(re.findall(r'[A-Z]+', sents)) for sents in my_train['comment_text']]

In [157]:
all_toxic = " ".join(my_train.loc[my_train['toxic'] == 1]['comment_text'])
all_severe_toxic = " ".join(my_train.loc[my_train['severe_toxic'] == 1]['comment_text'])
all_obscene = " ".join(my_train.loc[my_train['obscene'] == 1]['comment_text'])
all_threat = " ".join(my_train.loc[my_train['threat'] == 1]['comment_text'])
all_insult = " ".join(my_train.loc[my_train['insult'] == 1]['comment_text'])
all_identity_hate = " ".join(my_train.loc[my_train['identity_hate'] == 1]['comment_text'])                                  

In [158]:
print('severe_toxic', sum(upper_lower_words('severe_toxic')))
print('toxic', sum(upper_lower_words('toxic')))
print('obscene', sum(upper_lower_words('obscene')))
print('threat', sum(upper_lower_words('threat')))
print('insult', sum(upper_lower_words('insult')))
print('identity_hate', sum(upper_lower_words('identity_hate')))

severe_toxic 19582
toxic 56533
obscene 36456
threat 5047
insult 31223
identity_hate 9179


In [159]:
print('severe_toxic', sum(upper_lower_words('severe_toxic'))/len(all_severe_toxic))
print('toxic', sum(upper_lower_words('toxic'))/len(all_toxic))
print('obscene', sum(upper_lower_words('obscene'))/len(all_obscene))
print('threat', sum(upper_lower_words('threat'))/len(all_threat))
print('insult', sum(upper_lower_words('insult'))/len(all_insult))
print('identity_hate', sum(upper_lower_words('identity_hate'))/len(all_identity_hate))

severe_toxic 0.01970680318456787
toxic 0.009388023988092638
obscene 0.01065063967819035
threat 0.02304155880916184
insult 0.010110011090154048
identity_hate 0.013745483188401023


In [160]:
# И так же стоит посмотреть на пунктуацию

In [161]:
def punkt_len(toxic_type):
  final_len = 0
  all_text = " ".join(my_train.loc[my_train[toxic_type] == 1]['comment_text'])
  all_punkt = re.findall(r'[\.,?!)\";}\]\*:@\'\({\[]+', all_text)
  for punkt_peace in all_punkt:
    final_len += len(punkt_peace)
  return final_len


In [162]:
print('toxic', punkt_len('toxic'))
print('severe_toxic', punkt_len('severe_toxic'))
print('obscene', punkt_len('obscene'))
print('threat', punkt_len('threat'))
print('insult', punkt_len('insult'))
print('identity_hate', punkt_len('identity_hate'))

toxic 269048
severe_toxic 51662
obscene 151698
threat 16543
insult 134153
identity_hate 25735


In [163]:
print('toxic', punkt_len('toxic')/len(all_toxic))
print('severe_toxic', punkt_len('severe_toxic')/len(all_severe_toxic))
print('obscene', punkt_len('obscene')/len(all_obscene))
print('threat', punkt_len('threat')/len(all_threat))
print('insult', punkt_len('insult')/len(all_insult))
print('identity_hate', punkt_len('identity_hate')/len(all_identity_hate))

toxic 0.04467884382481645
severe_toxic 0.051991260653720006
obscene 0.0443186509189741
threat 0.07552536306319879
insult 0.0434387572551464
identity_hate 0.03853796817229549


## **Бейзлайн модель из sklearn (векторайзер + модель) с отбором признаков (через l1 регуляризацию, на глаз через анализ важных параметров или через permutation importance)**

In [164]:
#https://github.com/mannefedov/hse_ml_m1/blob/master/1_intro_regression/pandas_regression.ipynb - регуляризация
# тут и статистики есть!!

#https://github.com/mannefedov/hse_ml_m1/blob/master/3_overfitting_validation/overfitting_validation.ipynb - permutation importance


#ансамбли - https://github.com/DmitryKutsev/ml_hw/blob/master/hw5.ipynb

#ох..

In [189]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [191]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import LinearSVR
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score, roc_curve
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import eli5
from eli5.sklearn import PermutationImportance
import warnings

In [206]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
X = cv.fit_transform(my_train['comment_text'])
y = my_train['toxic']
miniX = cv.fit_transform(my_train['comment_text'][:100])
miniy = my_train['toxic'][:100]

In [169]:
# X = my_train['comment_text']
# y = my_train['toxic']

In [187]:

def scores(model):
  ''' Функция для оценки моделей на тесте'''
  
  predicted = model.predict(X_test)
  acc = accuracy_score(predicted, twenty_test.target)
  micro_f1 = f1_score(predicted, twenty_test.target, average = 'micro')
  micro_p = precision_score(predicted, twenty_test.target, average = 'micro')
  micro_r = recall_score(predicted, twenty_test.target, average = 'micro')
  macro_f1 = f1_score(predicted, twenty_test.target, average = 'macro')
  macro_p = precision_score(predicted, twenty_test.target, average = 'macro')
  macro_r = recall_score(predicted, twenty_test.target, average = 'macro')
  print('acc={0:1.4f}'.format(acc))
  print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}'.format(micro_f1, micro_p, micro_r))
  print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}\n'.format(macro_f1, macro_p, macro_r))

In [207]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
miniX_train, miniX_test, miniy_train, miniy_test = train_test_split(miniX, miniy, random_state=42)

In [181]:
regressor = Lasso(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
mean_absolute_error(y_test, preds)

0.17290650702074034

In [200]:
lr = LogisticRegression()

In [186]:
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 100.0, 'penalty': 'l2'}
accuracy : 0.9367891240100217


In [ ]:
# если использовать много параметров, то грид обучается слишком долго, оставил самый минимум 
# для экономии времени
parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.001, 0.01, 0.08, 0.1, 0.15, 1.0, 10.0, 100.0],
                  'max_iter': [2,10,50,100,1000],
                  'multi_class' : ['auto', 'ovr', 'multinomial'],
                  'tol':[0.0001, 0.001],
                  'l1_ratio': [0, 1],
                  'verbose': [0, 1],
                  'intercept_scaling': [1, 2]
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=10, scoring='roc_auc')
grid_search.fit(miniX_train, miniy_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

In [209]:
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.9857142857142858
Best parameters: {'C': 1.0, 'class_weight': 'balanced', 'intercept_scaling': 1, 'l1_ratio': 1, 'max_iter': 2, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.001, 'verbose': 0}
